# New Version

First, try sklearn

In [ ]:
import sklearn


In [ ]:
import re


In [ ]:
import numpy as np

In [ ]:
from sklearn import cluster

In [ ]:
LIB = sklearn

In [189]:
'AsadSLF'.lower()

'asadslf'

In [190]:


def split_func_name(func):
    new_str = ''
    for i, l in enumerate(func):
        if i>0 and l.isupper() and func[i-1].islower():
            new_str+='.'
        new_str+=l
    return re.split('\.|_', new_str.lower())

In [ ]:
with open('/projects/bdata/jupyter/decision_points/{}.txt'.format(LIB.__name__),'r') as f:
    dp  = f.read().split('\n')

In [248]:
with open('./file_funcs.json','r') as f:
    data = json.load(f)

In [ ]:
dp = [f for f in dp if f!='']

In [ ]:
vocab = []
doc_frequency = {}
for f in dp:
    tokens = split_func_name(f)
#     tokens = 
    for t in tokens:
        if t not in doc_frequency:
            doc_frequency[t] = 0
        doc_frequency[t]+=1
        if t not in vocab:
            vocab.append(t)
        

In [ ]:
len(vocab)

In [ ]:
doc_frequency

In [ ]:
word2idx = {v:i for i,v in enumerate(vocab)}

func2vector = {}
for f in dp:
    func2vector[f] = np.zeros(len(vocab))
    for t in split_func_name(f):
        func2vector[f][word2idx[t]]=1*np.log(len(dp)/doc_frequency[t])

In [ ]:
func2vector

In [ ]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [ ]:
cooccur_matrix

In [ ]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [ ]:
a

In [ ]:
len(dp)

In [92]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.999, affinity="precomputed", linkage="average").fit(a)

In [93]:
model.n_clusters_

5

In [94]:
clusters = {}
for i, l in enumerate(model.labels_):
    if l not in clusters:
        clusters[l] = []
    clusters[l].append(idx2func[i])
clusters = list(clusters.values())

In [101]:
root = {}
# cur_root = root
for f in dp:
    cur_root = root
    for t in f.split('.'):
        if t not in cur_root:
            cur_root[t] = {}
        cur_root = cur_root[t]

In [102]:
print(json.dumps(root,indent=2))

{
  "sklearn": {
    "base": {
      "clone": {},
      "is_classifier": {}
    },
    "calibration": {
      "CalibratedClassifierCV": {},
      "calibration_curve": {}
    },
    "clone": {},
    "cluster": {
      "AffinityPropagation": {},
      "AgglomerativeClustering": {},
      "Birch": {},
      "DBSCAN": {},
      "FeatureAgglomeration": {},
      "KMeans": {},
      "MeanShift": {},
      "MiniBatchKMeans": {},
      "OPTICS": {},
      "SpectralBiclustering": {},
      "SpectralClustering": {},
      "SpectralCoclustering": {},
      "affinity_propagation": {},
      "bicluster": {
        "SpectralBiclustering": {},
        "SpectralCoclustering": {}
      },
      "birch": {
        "Birch": {}
      },
      "estimate_bandwidth": {},
      "hierarchical": {
        "AgglomerativeClustering": {}
      },
      "k_means": {},
      "spectral_clustering": {}
    },
    "covariance": {
      "EllipticEnvelope": {},
      "EmpiricalCovariance": {},
      "GraphLasso": {},
   

In [115]:
freq_root = []
threshold = 4
def traverse_tree(prefix, root):
    for k in root:
        cur_prefix = '{}.{}'.format(prefix, k) if prefix!='' else k
        if len(root[k])>threshold:
            freq_root.append(cur_prefix)
        traverse_tree(cur_prefix, root[k])
            

In [116]:
traverse_tree('', root)

In [119]:
freq_root = [r for r in freq_root if r!=LIB.__name__]

In [120]:
len(freq_root)

20

In [247]:
# for i, r in enumerate(freq_root):
#     print('-'*20)
r = freq_root[20]
funcs = [f[len(r)+1:] for f in dp if f.startswith(r)]


# for f in f

vocab = []
doc_frequency = {}
for f in funcs:
#     print(f)
    tokens = split_func_name(f)
#     tokens = 
    for t in tokens:
        if t not in doc_frequency:
            doc_frequency[t] = 0
        doc_frequency[t]+=1
        if t not in vocab:
            vocab.append(t)
        
word2idx = {v:i for i,v in enumerate(vocab)}


func2vector = {}
for f in funcs:
    func2vector[f] = np.zeros(len(vocab))
    for t in split_func_name(f):
        func2vector[f][word2idx[t]]=1
            
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)
a = 1- cooccur_matrix/cooccur_matrix.max()
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.8, affinity="precomputed", linkage="average").fit(a)
print(model.n_clusters_)
    
clusters = {}
for i, l in enumerate(model.labels_):
    if l not in clusters:
        clusters[l] = []
    clusters[l].append(idx2func[i])
clusters = list(clusters.values())
for c in clusters:
    print('-'*20)
#     print(c)
    for f in c:
        print('{}.{}'.format(r, f))

IndexError: list index out of range

In [319]:
# for i, r in enumerate(freq_root):
#     print('-'*20)
r = freq_root[19]
funcs = [f for f in dp if f.startswith(r)]



func2vector = {}
for f in funcs:
    func2vector[f] = np.zeros(len(data))
    
idx2file = list(data.keys())
file2idx = {f:i for i, f in enumerate(idx2file)}
    
for d in data:
    for f in set(data[d]["funcs"]):
        if f in func2vector:
            func2vector[f][file2idx[d]]=1
    
    
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)
a = 1- cooccur_matrix/cooccur_matrix.max()
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=1, affinity="precomputed", linkage="average").fit(a)
print(model.n_clusters_)
    
clusters = {}
for i, l in enumerate(model.labels_):
    if l not in clusters:
        clusters[l] = []
    clusters[l].append(idx2func[i])
clusters = list(clusters.values())
for c in clusters:
    print('-'*20)
    for f in c:
        print('{}.{}'.format(r, f))

5
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.cartesian
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.density
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.randomized_svd
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.softmax
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.weighted_mode


___

# Calculate top n instead of clustering 

先用 common sub token 试试 top n

In [1]:
from utils import split_func_name

In [9]:
import numpy as np

In [13]:
import random

In [16]:
import scipy

In [18]:
from scipy import *

In [20]:
import scipy.spatial

In [25]:
import sklearn.metrics

In [2]:
with open('/projects/bdata/jupyter/decision_points/sklearn.txt','r') as f:
    dp = f.read().split('\n')
    dp = [l for l in dp if l]
    


In [3]:
dp

['sklearn.base.clone',
 'sklearn.base.is_classifier',
 'sklearn.calibration.CalibratedClassifierCV',
 'sklearn.calibration.calibration_curve',
 'sklearn.clone',
 'sklearn.cluster.AffinityPropagation',
 'sklearn.cluster.AgglomerativeClustering',
 'sklearn.cluster.Birch',
 'sklearn.cluster.DBSCAN',
 'sklearn.cluster.FeatureAgglomeration',
 'sklearn.cluster.KMeans',
 'sklearn.cluster.MeanShift',
 'sklearn.cluster.MiniBatchKMeans',
 'sklearn.cluster.OPTICS',
 'sklearn.cluster.SpectralBiclustering',
 'sklearn.cluster.SpectralClustering',
 'sklearn.cluster.SpectralCoclustering',
 'sklearn.cluster.affinity_propagation',
 'sklearn.cluster.bicluster.SpectralBiclustering',
 'sklearn.cluster.bicluster.SpectralCoclustering',
 'sklearn.cluster.birch.Birch',
 'sklearn.cluster.estimate_bandwidth',
 'sklearn.cluster.hierarchical.AgglomerativeClustering',
 'sklearn.cluster.k_means',
 'sklearn.cluster.spectral_clustering',
 'sklearn.covariance.EllipticEnvelope',
 'sklearn.covariance.EmpiricalCovariance'

In [4]:
len(dp)

433

In [245]:
vocab = []
for f in dp:
    tokens = split_func_name(f)
    for t in tokens:
        if t not in vocab:
            vocab.append(t)
w2i = {w:i for i, w in enumerate(vocab)}

In [246]:
len(vocab)

441

In [247]:
w2i

{'sklearn': 0,
 'base': 1,
 'clone': 2,
 'is': 3,
 'classifier': 4,
 'calibration': 5,
 'calibrated': 6,
 'cv': 7,
 'curve': 8,
 'cluster': 9,
 'affinity': 10,
 'propagation': 11,
 'agglomerative': 12,
 'clustering': 13,
 'birch': 14,
 'dbscan': 15,
 'feature': 16,
 'agglomeration': 17,
 'kmeans': 18,
 'mean': 19,
 'shift': 20,
 'mini': 21,
 'batch': 22,
 'optics': 23,
 'spectral': 24,
 'biclustering': 25,
 'coclustering': 26,
 'bicluster': 27,
 'estimate': 28,
 'bandwidth': 29,
 'hierarchical': 30,
 'k': 31,
 'means': 32,
 'covariance': 33,
 'elliptic': 34,
 'envelope': 35,
 'empirical': 36,
 'graph': 37,
 'lasso': 38,
 'graphical': 39,
 'ledoit': 40,
 'wolf': 41,
 'min': 42,
 'cov': 43,
 'det': 44,
 'oas': 45,
 'shrunk': 46,
 'cross': 47,
 'decomposition': 48,
 'cca': 49,
 'plscanonical': 50,
 'plsregression': 51,
 'plssvd': 52,
 'datasets': 53,
 'bunch': 54,
 'california': 55,
 'housing': 56,
 'fetch': 57,
 'clear': 58,
 'data': 59,
 'home': 60,
 'dump': 61,
 'svmlight': 62,
 'file'

In [338]:
func2vector = {}
for f in dp:
    func2vector[f] = np.zeros(len(vocab))
    tokens = split_func_name(f)
    for i, t in enumerate(tokens):
        func2vector[f][w2i[t]]=1*i/len(tokens)

In [339]:
f = random.choice(dp)

In [340]:
f

'sklearn.ensemble.RandomForestClassifier'

In [341]:
vectors = np.stack([func2vector[f] for f in dp])

In [342]:
sim_mat = sklearn.metrics.pairwise.cosine_similarity(vectors)

In [343]:
sim_mat

array([[1.        , 0.11952286, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.11952286, 1.        , 0.43915503, ..., 0.        , 0.28829999,
        0.        ],
       [0.        , 0.43915503, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.34465617,
        0.34465617],
       [0.        , 0.28829999, 0.        , ..., 0.34465617, 1.        ,
        0.25454545],
       [0.        , 0.        , 0.        , ..., 0.34465617, 0.25454545,
        1.        ]])

In [344]:
sorted_mat = np.argsort(sim_mat, axis = 1)

In [345]:
sorted_mat[:,-10:]

array([[144, 145, 146, ..., 128,   4,   0],
       [338, 131, 411, ..., 126, 122,   1],
       [198, 203,  30, ..., 216, 218,   2],
       ...,
       [335, 418, 245, ..., 416, 417, 430],
       [335, 418, 245, ...,   1, 430, 431],
       [140, 139, 417, ..., 430, 418, 432]])

In [346]:
dist = 1-sim_mat

In [347]:
dist.shape

(433, 433)

In [352]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.7, affinity="precomputed", linkage="average").fit(dist)

In [353]:
model.n_clusters_

133

In [354]:
new_clusters = {}
# new_clusters = [[]]*model.n_clusters_
for i, l in enumerate(model.labels_):
    if l not in new_clusters:
        new_clusters[l] = []
    new_clusters[l].append(dp[i])
#     print(i)
clusters = list(new_clusters.values())

In [355]:
for c in clusters:
    print('-'*20)
    for f in c:
        print(f)

--------------------
sklearn.base.clone
sklearn.clone
--------------------
sklearn.base.is_classifier
sklearn.dummy.DummyClassifier
sklearn.ensemble.AdaBoostClassifier
sklearn.ensemble.BaggingClassifier
sklearn.ensemble.ExtraTreesClassifier
sklearn.ensemble.GradientBoostingClassifier
sklearn.ensemble.RandomForestClassifier
sklearn.ensemble.VotingClassifier
sklearn.ensemble.forest.RandomForestClassifier
sklearn.ensemble.gradient_boosting.GradientBoostingClassifier
sklearn.gaussian_process.GaussianProcessClassifier
sklearn.linear_model.PassiveAggressiveClassifier
sklearn.multiclass.OneVsOneClassifier
sklearn.multiclass.OneVsRestClassifier
sklearn.multiclass.OutputCodeClassifier
sklearn.multioutput.MultiOutputClassifier
sklearn.neighbors.KNeighborsClassifier
sklearn.neighbors.RadiusNeighborsClassifier
sklearn.tree.DecisionTreeClassifier
sklearn.tree.ExtraTreeClassifier
sklearn.tree.tree.DecisionTreeClassifier
--------------------
sklearn.calibration.CalibratedClassifierCV
sklearn.covarian

In [81]:
from sklearn import cluster

In [98]:
with open('./file_funcs.json','r') as f:
    data = json.load(f)

In [160]:
func2vector = {}
vector_size = len(data)
files = list(data.keys())
err_files = []
sklearn_counter = {}
for i, nb in enumerate(files):
    if i%10000 == 0:
        print('Log: {} notebooks processed'.format(i))
    funcs = []
    if nb in data:
        funcs = data[nb]["funcs"]
        linenos = data[nb]["linenos"]
    funcs = [func for func in funcs if func in dp]
    
    funcs = set(funcs)
    for func in funcs:
        if func not in func2vector:
            func2vector[func] = np.zeros(vector_size)
        if func not in sklearn_counter:
            sklearn_counter[func]=0
        func2vector[func][i] = 1
        sklearn_counter[func]+=1

Log: 0 notebooks processed
Log: 10000 notebooks processed
Log: 20000 notebooks processed
Log: 30000 notebooks processed
Log: 40000 notebooks processed
Log: 50000 notebooks processed
Log: 60000 notebooks processed
Log: 70000 notebooks processed
Log: 80000 notebooks processed


In [128]:
f_map = {}
for f in sklearn_counter:
    for root in dp:
        if f.startswith(root):
            f_map[f] = root

In [129]:
f_map

{'sklearn.feature_extraction.text.TfidfVectorizer.transform.multiply': 'sklearn.feature_extraction.text.TfidfVectorizer',
 'sklearn.feature_extraction.text.TfidfVectorizer.transform': 'sklearn.feature_extraction.text.TfidfVectorizer',
 'sklearn.feature_extraction.text.TfidfVectorizer': 'sklearn.feature_extraction.text.TfidfVectorizer',
 'sklearn.feature_extraction.text.TfidfVectorizer.fit': 'sklearn.feature_extraction.text.TfidfVectorizer',
 'sklearn.feature_extraction.text.TfidfVectorizer.transform.multiply.sum': 'sklearn.feature_extraction.text.TfidfVectorizer',
 'sklearn.linear_model.LogisticRegression': 'sklearn.linear_model.LogisticRegression',
 'sklearn.feature_extraction.text.CountVectorizer.transform': 'sklearn.feature_extraction.text.CountVectorizer',
 'sklearn.feature_extraction.text.CountVectorizer': 'sklearn.feature_extraction.text.CountVectorizer',
 'sklearn.feature_extraction.text.CountVectorizer.fit': 'sklearn.feature_extraction.text.CountVectorizer',
 'sklearn.svm.SVC':

In [131]:
for f in dp:
    if f not in func2vector:
        func2vector[f] = np.zeros(vector_size)

In [161]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [162]:
sim_mat = sklearn.metrics.pairwise.cosine_similarity(occur_matrix)

In [228]:
sorted_mat = np.argsort(sim_mat, axis = 1)

In [236]:
for i, s in enumerate(sorted_mat[:,-10:]):
    print('-'*20)
    for j in s[::-1]:
        if sim_mat[i,j]>=0.44:
            print(idx2func[j])

--------------------
sklearn.feature_extraction.text.TfidfVectorizer
--------------------
sklearn.feature_extraction.text.CountVectorizer
--------------------
sklearn.linear_model.LogisticRegression
--------------------
sklearn.svm.SVC
--------------------
sklearn.metrics.mean_squared_error
--------------------
sklearn.metrics.classification_report
sklearn.metrics.confusion_matrix
--------------------
sklearn.preprocessing.LabelEncoder
--------------------
sklearn.metrics.confusion_matrix
sklearn.metrics.classification_report
--------------------
sklearn.linear_model.Ridge
sklearn.linear_model.Lasso
--------------------
sklearn.linear_model.Lasso
sklearn.linear_model.Ridge
--------------------
sklearn.neural_network.MLPClassifier
--------------------
sklearn.preprocessing.StandardScaler
--------------------
sklearn.preprocessing.LabelBinarizer
--------------------
sklearn.cluster.k_means
--------------------
sklearn.cluster.spectral_clustering
sklearn.feature_extraction.image.img_to_gr

In [163]:
sim_mat.shape

(390, 390)

In [138]:
len(idx2func)

433

In [164]:
a = 1- sim_mat

In [241]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.44, affinity="precomputed", linkage="average").fit(a)





In [242]:
model.n_clusters_

350

In [243]:
clusters = {}
for i, l in enumerate(model.labels_):
    if l not in clusters:
        clusters[l] = []
    clusters[l].append(idx2func[i])
clusters = list(clusters.values())

In [244]:
for c in clusters:
    print('-'*20)
    for f in c:
        print(f)

--------------------
sklearn.feature_extraction.text.TfidfVectorizer
--------------------
sklearn.feature_extraction.text.CountVectorizer
--------------------
sklearn.linear_model.LogisticRegression
--------------------
sklearn.svm.SVC
--------------------
sklearn.metrics.mean_squared_error
--------------------
sklearn.metrics.classification_report
--------------------
sklearn.preprocessing.LabelEncoder
--------------------
sklearn.metrics.confusion_matrix
--------------------
sklearn.linear_model.Ridge
sklearn.linear_model.Lasso
--------------------
sklearn.neural_network.MLPClassifier
--------------------
sklearn.preprocessing.StandardScaler
--------------------
sklearn.preprocessing.LabelBinarizer
--------------------
sklearn.cluster.k_means
--------------------
sklearn.cluster.spectral_clustering
sklearn.feature_extraction.image.img_to_graph
--------------------
sklearn.metrics.accuracy_score
--------------------
sklearn.metrics.log_loss
--------------------
sklearn.preprocessing.O

# Read Similar Sets from .txt

In [1]:
import pdb

In [2]:
with open('./cooccurence_alternatives.txt','r', encoding='utf-8-sig') as f:
    lines = f.read().split('\n')
    lines = [l for l in lines if l]

cooccur_sets = []
for l in lines:
    
    if l.startswith('-'):
        cooccur_sets.append([])
    else:
        cooccur_sets[-1].append(l)

cooccur_f2s = {}
for i, s in enumerate(cooccur_sets):
    for f in s:
        cooccur_f2s[f] = i

In [3]:
cooccur_sets

[['sklearn.cluster.AffinityPropagation',
  'sklearn.cluster.AgglomerativeClustering',
  'sklearn.cluster.Birch',
  'sklearn.cluster.DBSCAN',
  'sklearn.cluster.KMeans',
  'sklearn.cluster.MeanShift',
  'sklearn.cluster.MiniBatchKMeans',
  'sklearn.cluster.SpectralClustering',
  'sklearn.cluster.estimate_bandwidth'],
 ['sklearn.cluster.FeatureAgglomeration'],
 ['sklearn.cluster.OPTICS'],
 ['sklearn.cluster.SpectralBiclustering'],
 ['sklearn.cluster.SpectralCoclustering'],
 ['sklearn.cluster.affinity_propagation'],
 ['sklearn.cluster.bicluster.SpectralBiclustering'],
 ['sklearn.cluster.bicluster.SpectralCoclustering'],
 ['sklearn.cluster.birch.Birch'],
 ['sklearn.cluster.hierarchical.AgglomerativeClustering'],
 ['sklearn.cluster.k_means'],
 ['sklearn.cluster.spectral_clustering'],
 ['sklearn.covariance.EllipticEnvelope',
  'sklearn.covariance.EmpiricalCovariance',
  'sklearn.covariance.GraphLassoCV',
  'sklearn.covariance.LedoitWolf',
  'sklearn.covariance.MinCovDet',
  'sklearn.covarian

In [4]:
with open('./subtoken_alternatives.txt','r', encoding='utf-8-sig') as f:
    lines = f.read().split('\n')
    lines = [l for l in lines if l]

subtoken_sets = []
for l in lines:
    
    if l.startswith('-'):
        subtoken_sets.append([])
    else:
        subtoken_sets[-1].append(l)

subtoken_f2s = {}
for i, s in enumerate(subtoken_sets):
    for f in s:
        subtoken_f2s[f] = i

In [13]:
similar_sets = []
visited = []
for s in cooccur_sets:
    if any([f in visited for f in s]):
        continue
#     当前没有sets 或者上一个set 超过10，加入新的set
    similar_sets.append(s)
#     如果新的set 大于10， 直接进入下一次循环，加入新的sets
    if len(similar_sets[-1])>=10:
        continue
#     不然就从 subtoken sets 中找补充的函数
    for f in similar_sets[-1]:
        supple_set = subtoken_sets[subtoken_f2s[f]]
        similar_sets[-1] = list(set(similar_sets[-1]+supple_set))
#         if len(similar_sets)>=10:
#             break
    visited+=similar_sets[-1]
#     pdb.set_trace()

In [14]:
similar_sets

[['sklearn.cluster.bicluster.SpectralBiclustering',
  'sklearn.cluster.SpectralClustering',
  'sklearn.cluster.AffinityPropagation',
  'sklearn.cluster.hierarchical.AgglomerativeClustering',
  'sklearn.cluster.SpectralCoclustering',
  'sklearn.cluster.bicluster.SpectralCoclustering',
  'sklearn.cluster.MiniBatchKMeans',
  'sklearn.cluster.SpectralBiclustering',
  'sklearn.cluster.KMeans',
  'sklearn.cluster.birch.Birch',
  'sklearn.cluster.DBSCAN',
  'sklearn.cluster.AgglomerativeClustering',
  'sklearn.cluster.Birch',
  'sklearn.cluster.MeanShift',
  'sklearn.cluster.estimate_bandwidth'],
 ['sklearn.cluster.FeatureAgglomeration'],
 ['sklearn.cluster.OPTICS'],
 ['sklearn.cluster.affinity_propagation'],
 ['sklearn.cluster.k_means'],
 ['sklearn.cluster.spectral_clustering'],
 ['sklearn.covariance.ledoit_wolf',
  'sklearn.covariance.GraphLasso',
  'sklearn.covariance.LedoitWolf',
  'sklearn.covariance.EmpiricalCovariance',
  'sklearn.covariance.GraphLassoCV',
  'sklearn.covariance.ShrunkC

In [15]:
for s in similar_sets:
    if len(s)==1:
        continue
    print('-'*20)
    for f in s:
        print(f)

--------------------
sklearn.cluster.bicluster.SpectralBiclustering
sklearn.cluster.SpectralClustering
sklearn.cluster.AffinityPropagation
sklearn.cluster.hierarchical.AgglomerativeClustering
sklearn.cluster.SpectralCoclustering
sklearn.cluster.bicluster.SpectralCoclustering
sklearn.cluster.MiniBatchKMeans
sklearn.cluster.SpectralBiclustering
sklearn.cluster.KMeans
sklearn.cluster.birch.Birch
sklearn.cluster.DBSCAN
sklearn.cluster.AgglomerativeClustering
sklearn.cluster.Birch
sklearn.cluster.MeanShift
sklearn.cluster.estimate_bandwidth
--------------------
sklearn.covariance.ledoit_wolf
sklearn.covariance.GraphLasso
sklearn.covariance.LedoitWolf
sklearn.covariance.EmpiricalCovariance
sklearn.covariance.GraphLassoCV
sklearn.covariance.ShrunkCovariance
sklearn.covariance.EllipticEnvelope
sklearn.covariance.OAS
sklearn.covariance.empirical_covariance
sklearn.covariance.MinCovDet
--------------------
sklearn.covariance.GraphicalLasso
sklearn.covariance.GraphicalLassoCV
--------------------

In [339]:
import pdb

In [322]:
print(lines[0])

﻿--------------------


_____

# Last Edited

In [ ]:
with open('./sklearn_dp.txt','r') as f:
    sklearn_dps = f.read().split()

In [ ]:
import json
with open('./func_counter.json','r') as f:
    func_counter = json.load(f)["func_counter"]
statsmodels_counter = {k: func_counter[k] for k in func_counter if k.startswith('sklearn.')}

In [ ]:
target_funcs = sklearn_dps

In [ ]:
statsmodels_counter

In [ ]:
sklearn_dps

In [ ]:
sklearn_roots = ['.'.join(f.split('.')[:-1]) for f in sklearn_dps]

In [ ]:
from importlib import import_module

In [ ]:
sklearn_dp  = sklearn_dps

In [ ]:
roots_counter = {}
for r in sklearn_roots:
    if r not in roots_counter:
        roots_counter[r]=0
    roots_counter[r]+=1
frequent_roots = {r: roots_counter[r] for r in roots_counter if roots_counter[r]>=5}
for r in frequent_roots:
#     if r=="statsmodels":continue
    try:
        mod = import_module(r)
#         print(mod.__all__)
        try:
            dps = ['{}.{}'.format(r, obj) for obj in mod.__all__] 
        except:
            dps =  ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
#         dps = ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
        sklearn_dp+=dps
#         print(dps)
    except Exception as e:
        print(e)
sklearn_dp = sorted(list(set(sklearn_dp)))

In [ ]:
len(sklearn_dp)

In [ ]:
funcs = [f for f in sklearn_dp if f.endswith("Regressor") or f.endswith("Classifier")]

In [ ]:
funcs

In [ ]:
target_funcs = funcs

In [ ]:
nb_path = '/home/gezhang/data/jupyter/target'
notebooks = os.listdir(nb_path)

In [ ]:
import os

In [ ]:
sklearn_notebooks = []
for i, nb in enumerate(notebooks):
    if i%10000==0:
        print(i)
    with open(os.path.join(nb_path, nb),'r') as f:
#         print('--')
        content = f.read()
    tokens = content.split()
    if 'sklearn' in tokens:
        sklearn_notebooks.append(nb)
    

In [ ]:
len(sklearn_notebooks)

In [ ]:
sklearn_notebooks = sklearn_notebooks[:20000]

In [ ]:
from extract_func import process_file

In [ ]:
func2vector = {}
vector_size = len(sklearn_notebooks)
err_files = []

In [ ]:
for f in target_funcs:
    func2vector[f] = np.zeros(vector_size)

In [ ]:
import numpy as np

In [ ]:
for i, nb in enumerate(sklearn_notebooks):
    if i%10000 == 0:
        print('Log: {} notebooks processed'.format(i))
    funcs = []
    try:
        funcs, linenos = process_file(os.path.join(nb_path, nb))
    except Exception as e:
        err_files.append(nb)
#     funcs = [func for func in funcs if func in no_loop_sklearn_funcs]
    for func in funcs:
        if func not in target_funcs:
            continue
#         if not func.startswith('sklearn'):
#             continue
#         if func not in func2vector:
#             func2vector[func] = np.zeros(vector_size)
        func2vector[func][i] = 1

In [ ]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [ ]:
cooccur_matrix.shape

In [ ]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [ ]:
from sklearn import cluster


In [ ]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4, affinity="precomputed", linkage="average").fit(a)

In [ ]:
clusters = []
n_samples = len(idx2func)
for child in model.children_:
    clusters.append([])
    for branch in child:
        if branch < n_samples:
            clusters[-1].append(idx2func[branch])
        else:
            clusters[-1]+=clusters[branch-n_samples]

# cooccurence

In [ ]:
for c in clusters:
    if len(c)<=10:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)


In [ ]:
split_func_name("sklearn.ensemble.AdaBoostClassifier")

In [ ]:
len(vocab)

In [ ]:
func2vector2

In [ ]:
clusters2 = []
n_samples2 = len(idx2func2)
for child in model2.children_:
    clusters2.append([])
    for branch in child:
        if branch < n_samples2:
            clusters2[-1].append(idx2func2[branch])
        else:
            clusters2[-1]+=clusters2[branch-n_samples2]

In [ ]:
clusters2

# Common Sub Token

In [ ]:
for c in clusters2:
    if len(c)<=15 and len(c)>3:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)

In [ ]:
occur_matrix2.shape

In [ ]:
occur_matrix.shape

In [ ]:
m = np.append(occur_matrix, occur_matrix2, axis = 1)

In [ ]:
model2 = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4).fit(m)

In [ ]:
clusters3 = []
n_samples2 = len(idx2func2)
for child in model2.children_:
    clusters3.append([])
    for branch in child:
        if branch < n_samples2:
            clusters3[-1].append(idx2func2[branch])
        else:
            clusters3[-1]+=clusters3[branch-n_samples2]

In [ ]:
clusters3

# sub token+ cooccur

In [ ]:
for c in clusters3:
    if len(c)<=15 and len(c)>3:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)